In [1]:
pip install numpy pandas matplotlib tensorflow keras scikit-learn vmdpy scipy

# For Better Accuracy

In [2]:
import os
import scipy.io as sio
import numpy as np

# Base dataset folder
data_path = "/content/drive/MyDrive/Ninapro sEMG Dataset"

def load_all_subjects_dynamic(data_path):
    emg_all, labels_all = [], []

    # Find all subject folders (like s1, s2, s3...)
    subject_folders = sorted([f for f in os.listdir(data_path) if f.lower().startswith("s")])

    for subj in subject_folders:
        subj_path = os.path.join(data_path, subj)

        # Find all E1–E3 files automatically
        mat_files = sorted([f for f in os.listdir(subj_path) if f.endswith(".mat")])

        for file_name in mat_files:
            file_path = os.path.join(subj_path, file_name)
            print(f"Loading {file_name}...")

            data = sio.loadmat(file_path)
            emg_all.append(data['emg'])
            labels_all.append(data['restimulus'])

    return np.vstack(emg_all), np.vstack(labels_all)

# 🔹 Load dataset (auto detects all subjects available)
emg_data, labels_data = load_all_subjects_dynamic(data_path)

print("EMG shape:", emg_data.shape)
print("Labels shape:", labels_data.shape)


Loading S1_A1_E1.mat...
Loading S1_A1_E2.mat...
Loading S1_A1_E3.mat...
Loading S10_A1_E1.mat...
Loading S10_A1_E2.mat...
Loading S10_A1_E3.mat...
Loading S2_A1_E1.mat...
Loading S2_A1_E2.mat...
Loading S2_A1_E3.mat...
Loading S3_A1_E1.mat...
Loading S3_A1_E2.mat...
Loading S3_A1_E3.mat...
Loading S4_A1_E1.mat...
Loading S4_A1_E2.mat...
Loading S4_A1_E3.mat...
Loading S5_A1_E1.mat...
Loading S5_A1_E2.mat...
Loading S5_A1_E3.mat...
Loading S6_A1_E1.mat...
Loading S6_A1_E2.mat...
Loading S6_A1_E3.mat...
Loading S7_A1_E1.mat...
Loading S7_A1_E2.mat...
Loading S7_A1_E3.mat...
Loading S8_A1_E1.mat...
Loading S8_A1_E2.mat...
Loading S8_A1_E3.mat...
Loading S9_A1_E1.mat...
Loading S9_A1_E2.mat...
Loading S9_A1_E3.mat...
EMG shape: (4612696, 10)
Labels shape: (4612696, 1)


# Overlap Segmentation, Normalization and Filtering

Because for normal segmentation validation accuracy was less

In [3]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import mode
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

# Parameters
window_size = 500
stride = 250  # 50% overlap

# Segmentation with overlap and aligned labels
overlap_segments = []
overlap_labels = []

for i in range(0, len(emg_data) - window_size, stride):
    overlap_segments.append(emg_data[i:i+window_size])
    seg_label = mode(labels_data[i:i+window_size], keepdims=True)[0][0]  # most frequent label
    overlap_labels.append(seg_label)

overlap_segments = np.array(overlap_segments)
overlap_labels = np.array(overlap_labels)

print("Overlap Segments shape:", overlap_segments.shape)

# Normalization (all 10 channels)
scaler = StandardScaler()
overlap_reshaped = overlap_segments.reshape(-1, overlap_segments.shape[-1])
overlap_scaled = scaler.fit_transform(overlap_reshaped)
overlap_scaled = overlap_scaled.reshape(overlap_segments.shape)

print("Overlap Segments (normalized) shape:", overlap_scaled.shape)

# Filter gestures 1–10
labels_clean_overlap = overlap_labels.flatten()
valid_indices_overlap = np.where((labels_clean_overlap >= 1) & (labels_clean_overlap <= 10))[0]

X_overlap = overlap_scaled[valid_indices_overlap]
y_overlap = labels_clean_overlap[valid_indices_overlap]

# One-hot encode labels
num_classes = len(np.unique(y_overlap))
y_overlap_cat = to_categorical(y_overlap - 1, num_classes=num_classes)

# Train/Test split
X_train_overlap, X_test_overlap, y_train_overlap, y_test_overlap = train_test_split(
    X_overlap, y_overlap_cat,
    test_size=0.2,
    random_state=42,
    stratify=y_overlap_cat
)

print("Overlap Train shape:", X_train_overlap.shape, y_train_overlap.shape)
print("Overlap Test shape:", X_test_overlap.shape, y_test_overlap.shape)


Overlap Segments shape: (18449, 500, 10)
Overlap Segments (normalized) shape: (18449, 500, 10)
Overlap Train shape: (3444, 500, 10) (3444, 10)
Overlap Test shape: (861, 500, 10) (861, 10)


# New CNN-LSTM Model

One epoch is taking around 80 seconds, whole process taken around 2 hours to execute.

It is taking a lot of time for execution because:

Increased data a lot s1 to s10.

Heavier CNN–LSTM, Conv1D layers: 64 → 128 → 256 filters. (more parameters)

Augmentation doubled dataset size. (train+aug)

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np

# Augmentation Function
def augment_emg(X):
    X_aug = X.copy()
    noise = np.random.normal(0, 0.02, X.shape)
    scale = np.random.uniform(0.9, 1.1, (X.shape[0], 1, X.shape[2]))
    shift = np.random.randint(-5, 5)
    X_aug = np.roll(X_aug, shift, axis=1)
    X_aug = X_aug * scale + noise
    return X_aug

# Apply augmentation
X_train_aug = augment_emg(X_train_overlap)
y_train_aug = y_train_overlap.copy()

# Combine original + augmented
X_train_final = np.vstack([X_train_overlap, X_train_aug])
y_train_final = np.vstack([y_train_overlap, y_train_aug])

print("✅ New training shape after augmentation:", X_train_final.shape)

# 🔹 Model
model_overlap = Sequential()

# CNN Block 1
model_overlap.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(500, 10)))
model_overlap.add(BatchNormalization())
model_overlap.add(MaxPooling1D(pool_size=2))
model_overlap.add(Dropout(0.3))

# CNN Block 2
model_overlap.add(Conv1D(128, kernel_size=4, activation='relu'))
model_overlap.add(BatchNormalization())
model_overlap.add(MaxPooling1D(pool_size=2))
model_overlap.add(Dropout(0.3))

# CNN Block 3
model_overlap.add(Conv1D(256, kernel_size=5, activation='relu'))
model_overlap.add(BatchNormalization())
model_overlap.add(MaxPooling1D(pool_size=4))
model_overlap.add(Dropout(0.4))

# LSTM Layer
model_overlap.add(LSTM(128, dropout=0.4, recurrent_dropout=0.3))

# Dense Layers
model_overlap.add(Dense(128, activation='relu'))
model_overlap.add(Dropout(0.4))
model_overlap.add(Dense(num_classes, activation='softmax'))

# Compile
model_overlap.compile(optimizer=Adam(learning_rate=0.0003),
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])

# Early stopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# Train
history_overlap = model_overlap.fit(
    X_train_final, y_train_final,
    validation_data=(X_test_overlap, y_test_overlap),
    epochs=80,
    batch_size=32,
    callbacks=[early_stop]
)


✅ New training shape after augmentation: (6888, 500, 10)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/80
216/216 ━━━━━━━━━━━━━━━━━━━━ 70s 286ms/step - accuracy: 0.1093 - loss: 2.3451 - val_accuracy: 0.1080 - val_loss: 2.3142
Epoch 2/80
216/216 ━━━━━━━━━━━━━━━━━━━━ 86s 307ms/step - accuracy: 0.1350 - loss: 2.2827 - val_accuracy: 0.1498 - val_loss: 2.2597
Epoch 3/80
216/216 ━━━━━━━━━━━━━━━━━━━━ 78s 289ms/step - accuracy: 0.1552 - loss: 2.2418 - val_accuracy: 0.1603 - val_loss: 2.2262
Epoch 4/80
 77/216 ━━━━━━━━━━━━━━━━━━━━ 40s 290ms/step - accuracy: 0.1926 - loss: 2.1781

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-1102329246.py", line 65, in <cell line: 0>
    history_overlap = model_overlap.fit(
                      ^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 117, in error_handler
    return fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py", line 371, in fit
    logs = self.train_function(iterator)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py", line 219, in function
    opt_outputs = multi_step_on_iterator(iterator)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tensorflow/python/util/trace

TypeError: object of type 'NoneType' has no len()

In [ ]:
test_loss, test_acc = model_overlap.evaluate(X_test_overlap, y_test_overlap)
print(f"🔥 Final Validation Accuracy: {test_acc*100:.2f}%")

# Confusion Matrix & Per-Class Accuracy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Predictions
y_pred_probs = model_overlap.predict(X_test_overlap)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test_overlap, axis=1)

# Confusion Matrix
cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[f"G{i}" for i in range(1, num_classes+1)],
            yticklabels=[f"G{i}" for i in range(1, num_classes+1)])
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix")
plt.show()

print("Classification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=[f"G{i}" for i in range(1, num_classes+1)]))


# Training vs Validation Accuracy & Loss

In [ ]:
import matplotlib.pyplot as plt

# Accuracy Plot
plt.figure(figsize=(10,4))
plt.plot(history_overlap.history['accuracy'], label='Train Accuracy')
plt.plot(history_overlap.history['val_accuracy'], label='Validation Accuracy')
plt.title("Training vs Validation Accuracy")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid(True)
plt.show()

# Loss Plot
plt.figure(figsize=(10,4))
plt.plot(history_overlap.history['loss'], label='Train Loss')
plt.plot(history_overlap.history['val_loss'], label='Validation Loss')
plt.title("Training vs Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Saving model for future use
model_overlap.save("sEMG_model.h5")

# Below are all the methods which I tried but  couldn't achieve desired results.

In [ ]:
# import scipy.io as sio
# import numpy as np
# from sklearn.preprocessing import StandardScaler

# # Load EMG & labels from MAT file
# data = sio.loadmat('/content/drive/MyDrive/Ninapro sEMG Dataset/s1/S1_A1_E1.mat')

# emg = data['emg']           # EMG signals
# labels = data['restimulus'] # Gesture labels

# print("EMG shape:", emg.shape)
# print("Labels shape:", labels.shape)

# # Parameters
# window_size = 500
# segments = []
# segment_labels = []

# # Create segments
# for i in range(0, len(emg) - window_size, window_size):
#     segments.append(emg[i:i+window_size])
#     segment_labels.append(labels[i+window_size//2])  # mid label

# segments = np.array(segments)
# segment_labels = np.array(segment_labels)

# # Normalize
# scaler = StandardScaler()
# segments_reshaped = segments.reshape(-1, segments.shape[-1])
# segments_scaled = scaler.fit_transform(segments_reshaped)
# segments_scaled = segments_scaled.reshape(segments.shape)

# print("Segments scaled shape:", segments_scaled.shape)


In [ ]:
# from vmdpy import VMD

# # Example: Take one EMG channel segment
# signal = segments_scaled[0,:,0]  # first segment, channel 0

# # VMD parameters
# alpha = 2000        # moderate bandwidth constraint
# tau = 0             # noise-tolerance (0 for noise-free)
# K = 5               # number of modes (IMFs)
# DC = 0              # no DC component
# init = 1            # initialize omegas uniformly
# tol = 1e-7

# # Run VMD
# u, u_hat, omega = VMD(signal, alpha, tau, K, DC, init, tol)

# print("Shape of IMFs:", u.shape)  # (K, length of signal)

Building CNN-LSTM Model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout

# model = Sequential()

# # CNN layers
# model.add(Conv1D(16, kernel_size=3, activation='relu', input_shape=(500, 3)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(32, kernel_size=4, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(64, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=4))

# # LSTM
# model.add(LSTM(100))

# # Dense layers
# model.add(Dense(64, activation='sigmoid'))
# model.add(Dropout(0.2))
# model.add(Dense(num_classes, activation='softmax'))

# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# model.summary()

In [ ]:
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_test, y_test),
#     epochs=30,
#     batch_size=16
# )

In [ ]:
# import scipy.io as sio
# import numpy as np

# data_path1 = "/content/drive/MyDrive/Ninapro sEMG Dataset/s1/"
# data_path2 = "/content/drive/MyDrive/Ninapro sEMG Dataset/s2/"

# files_s1 = [f"{data_path1}S1_A1_E1.mat", f"{data_path1}S1_A1_E2.mat", f"{data_path1}S1_A1_E3.mat"]
# files_s2 = [f"{data_path2}S2_A1_E1.mat", f"{data_path2}S2_A1_E2.mat", f"{data_path2}S2_A1_E3.mat"]

# def load_emg_multiple_subjects(file_groups):
#     emg_all = []
#     labels_all = []

#     for files in file_groups:
#         for f in files:
#             print(f"Loading {f}...")
#             data = sio.loadmat(f)
#             emg_all.append(data['emg'])
#             labels_all.append(data['restimulus'])

#     return np.vstack(emg_all), np.vstack(labels_all)

# emg_data, labels_data = load_emg_multiple_subjects([files_s1, files_s2])

# print("Combined EMG shape:", emg_data.shape)
# print("Combined Labels shape:", labels_data.shape)


# Segmentation

In [ ]:
# from sklearn.preprocessing import StandardScaler
# import numpy as np

# # Parameters
# window_size = 500
# segments = []
# segment_labels = []

# # Segment EMG data
# for i in range(0, len(emg_data) - window_size, window_size):
#     segments.append(emg_data[i:i+window_size])
#     segment_labels.append(labels_data[i+window_size//2])

# segments = np.array(segments)
# segment_labels = np.array(segment_labels)

# print("Segments shape:", segments.shape)
# print("Labels shape:", segment_labels.shape)


# Normalization

In [ ]:
# # Normalize segments
# scaler = StandardScaler()
# segments_reshaped = segments.reshape(-1, segments.shape[-1])
# segments_scaled = scaler.fit_transform(segments_reshaped)
# segments_scaled = segments_scaled.reshape(segments.shape)

# print("Segments scaled shape:", segments_scaled.shape)


# VMD Decompostion

In [ ]:
# from vmdpy import VMD

# alpha, tau, K, DC, init, tol = 2000, 0, 5, 0, 1, 1e-7
# imf_segments = []
# labels_for_imf = []

# # Apply VMD on each segment
# for idx, seg in enumerate(segments_scaled):
#     signal = seg[:, 0]  # Use Channel 0
#     u, _, _ = VMD(signal, alpha, tau, K, DC, init, tol)
#     selected_imfs = u[1:4]  # IMF2–IMF4
#     imf_segments.append(selected_imfs.T)
#     labels_for_imf.append(segment_labels[idx])

# imf_segments = np.array(imf_segments)
# labels_for_imf = np.array(labels_for_imf)

# print("IMF segments shape:", imf_segments.shape)


# Filter Valid Gestures

In [ ]:
# from tensorflow.keras.utils import to_categorical

# labels_clean = labels_for_imf.flatten()
# valid_indices = np.where((labels_clean >= 1) & (labels_clean <= 10))[0]

# X_valid = imf_segments[valid_indices]
# y_valid = labels_clean[valid_indices]

# num_classes = len(np.unique(y_valid))
# y_categorical = to_categorical(y_valid - 1, num_classes=num_classes)

# print("Final dataset shape:", X_valid.shape, y_categorical.shape)

# Train-Test Split

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(
#     X_valid, y_categorical, test_size=0.2, random_state=42, stratify=y_categorical
# )

# print("Train set shape:", X_train.shape, y_train.shape)
# print("Test set shape:", X_test.shape, y_test.shape)


# CNN-LSTM Model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam

# # Build CNN–LSTM Model
# model = Sequential()

# # CNN Layers
# model.add(Conv1D(16, kernel_size=3, activation='relu', input_shape=(500, 3)))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(32, kernel_size=4, activation='relu'))
# model.add(MaxPooling1D(pool_size=2))
# model.add(Conv1D(64, kernel_size=5, activation='relu'))
# model.add(MaxPooling1D(pool_size=4))

# # LSTM Layer
# model.add(LSTM(100))

# # Fully Connected Layers
# model.add(Dense(64, activation='sigmoid'))
# model.add(Dropout(0.3))  # slightly higher dropout for regularization
# model.add(Dense(num_classes, activation='softmax'))

# # Compile Model
# model.compile(optimizer=Adam(learning_rate=0.0005),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# # Model Summary
# model.summary()

In [ ]:
# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_test, y_test),
#     epochs=30,           # you can increase to 50 if accuracy is improving
#     batch_size=32
# )

In [ ]:
# print("Train X shape:", X_train.shape)
# print("Train Y shape:", y_train.shape)
# print("Unique gesture labels:", np.unique(np.argmax(y_train, axis=1)))


In [ ]:
# import numpy as np
# unique, counts = np.unique(np.argmax(y_train, axis=1), return_counts=True)
# print("Class distribution:", dict(zip(unique, counts)))


In [ ]:
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.utils import to_categorical

# # Filter valid gestures (1–10)
# labels_clean_raw = segment_labels.flatten()
# valid_indices_raw = np.where((labels_clean_raw >= 1) & (labels_clean_raw <= 10))[0]

# X_raw = segments_scaled[valid_indices_raw]
# y_raw = labels_clean_raw[valid_indices_raw]

# # One-hot encode labels
# num_classes_raw = len(np.unique(y_raw))
# y_categorical_raw = to_categorical(y_raw - 1, num_classes=num_classes_raw)

# # Train-Test Split
# X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
#     X_raw, y_categorical_raw, test_size=0.2, random_state=42, stratify=y_categorical_raw
# )

# print("Raw Train shape:", X_train_raw.shape)
# print("Raw Test shape:", X_test_raw.shape)

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam

# model_raw = Sequential()

# # CNN Layers
# model_raw.add(Conv1D(16, kernel_size=3, activation='relu', input_shape=(500, X_train_raw.shape[2])))
# model_raw.add(MaxPooling1D(pool_size=2))
# model_raw.add(Conv1D(32, kernel_size=4, activation='relu'))
# model_raw.add(MaxPooling1D(pool_size=2))
# model_raw.add(Conv1D(64, kernel_size=5, activation='relu'))
# model_raw.add(MaxPooling1D(pool_size=4))

# # LSTM Layer
# model_raw.add(LSTM(100))

# # Dense Layers
# model_raw.add(Dense(64, activation='sigmoid'))
# model_raw.add(Dropout(0.3))
# model_raw.add(Dense(num_classes_raw, activation='softmax'))

# model_raw.compile(optimizer=Adam(learning_rate=0.0005),
#                   loss='categorical_crossentropy',
#                   metrics=['accuracy'])

# model_raw.summary()

# history_raw = model_raw.fit(
#     X_train_raw, y_train_raw,
#     validation_data=(X_test_raw, y_test_raw),
#     epochs=30,
#     batch_size=32
# )


# Variational Mode Decomposition

In [ ]:
# from vmdpy import VMD

# # VMD parameters (tuned closer to paper)
# alpha = 2000       # Bandwidth constraint
# tau = 0            # Noise-tolerance (0 for noise-free)
# K = 5              # Number of modes (IMFs)
# DC = 0             # No DC component
# init = 1           # Initialize omegas uniformly
# tol = 1e-7

# imf_segments = []
# labels_for_imf = []

# print("Running VMD on all segments...")

# for idx, seg in enumerate(segments_scaled):
#     signal = seg[:, 0]  # Channel 0 for now (we can add multi-channel later)
#     u, _, _ = VMD(signal, alpha, tau, K, DC, init, tol)

#     # Select IMF2–IMF4
#     selected_imfs = u[1:4]  # Shape: (3, length)
#     imf_segments.append(selected_imfs.T)  # Transpose to (length, 3)
#     labels_for_imf.append(segment_labels[idx])

# imf_segments = np.array(imf_segments)
# labels_for_imf = np.array(labels_for_imf)

# print("✅ IMF segments shape:", imf_segments.shape)
# print("✅ IMF labels shape:", labels_for_imf.shape)


# Filter Valid Gestures and Train/Test Split

In [ ]:
# from tensorflow.keras.utils import to_categorical
# from sklearn.model_selection import train_test_split

# # Flatten labels
# labels_clean = labels_for_imf.flatten()

# # Keep only gestures 1–10
# valid_indices = np.where((labels_clean >= 1) & (labels_clean <= 10))[0]

# X_vmd = imf_segments[valid_indices]
# y_vmd = labels_clean[valid_indices]

# # One-hot encode labels
# num_classes = len(np.unique(y_vmd))
# y_vmd_categorical = to_categorical(y_vmd - 1, num_classes=num_classes)

# # Train-Test split
# X_train_vmd, X_test_vmd, y_train_vmd, y_test_vmd = train_test_split(
#     X_vmd, y_vmd_categorical,
#     test_size=0.2,
#     random_state=42,
#     stratify=y_vmd_categorical
# )

# print("✅ VMD Train shape:", X_train_vmd.shape, y_train_vmd.shape)
# print("✅ VMD Test shape:", X_test_vmd.shape, y_test_vmd.shape)


In [ ]:
# import matplotlib.pyplot as plt

# plt.figure(figsize=(10,5))
# plt.plot(X_train[0])   # One gesture
# plt.plot(X_train[50])  # Another gesture
# plt.legend(["IMF2","IMF3","IMF4"])
# plt.title("IMF comparison between two gestures")
# plt.show()


# VMD Parameter Tuning

In [ ]:
# # Tuned VMD parameters (based on EMG studies)
# alpha = 1000       # lower bandwidth → better separation
# tau = 0
# K = 6              # more modes → better IMF diversity
# DC = 0
# init = 1
# tol = 1e-7

# imf_segments_tuned = []
# labels_for_imf_tuned = []

# print("Running VMD with tuned parameters...")

# for idx, seg in enumerate(segments_scaled):
#     signal = seg[:, 0]  # Channel 0 for now
#     u, _, _ = VMD(signal, alpha, tau, K, DC, init, tol)

#     # Select IMF2–IMF4 (more distinct now)
#     selected_imfs = u[1:4]
#     imf_segments_tuned.append(selected_imfs.T)
#     labels_for_imf_tuned.append(segment_labels[idx])

# imf_segments_tuned = np.array(imf_segments_tuned)
# labels_for_imf_tuned = np.array(labels_for_imf_tuned)

# print("✅ Tuned IMF shape:", imf_segments_tuned.shape)


# CNN-LSTM Model

In [ ]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv1D, MaxPooling1D, BatchNormalization, LSTM, Dense, Dropout
# from tensorflow.keras.optimizers import Adam
# from tensorflow.keras.callbacks import EarlyStopping

# model = Sequential()

# # CNN Block 1
# model.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(500, 10)))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.3))

# # CNN Block 2
# model.add(Conv1D(128, kernel_size=4, activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=2))
# model.add(Dropout(0.3))

# # CNN Block 3
# model.add(Conv1D(256, kernel_size=5, activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling1D(pool_size=4))
# model.add(Dropout(0.3))

# # LSTM
# model.add(LSTM(128, dropout=0.3, recurrent_dropout=0.2))

# # Dense Layers
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.4))
# model.add(Dense(10, activation='softmax'))

# # Compile
# model.compile(optimizer=Adam(learning_rate=0.0003),
#               loss='categorical_crossentropy',
#               metrics=['accuracy'])

# model.summary()


Early Stopping

In [ ]:
# early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# history = model.fit(
#     X_train, y_train,
#     validation_data=(X_test, y_test),
#     epochs=80,
#     batch_size=32,
#     callbacks=[early_stop]
# )

Augmentation to fix overfitting

In [ ]:
# def augment_emg(X):
#     X_aug = X.copy()

#     # Add Gaussian Noise
#     noise = np.random.normal(0, 0.02, X.shape)
#     X_aug += noise

#     # Scale
#     scale = np.random.uniform(0.9, 1.1, (X.shape[0], 1, X.shape[2]))
#     X_aug *= scale

#     return X_aug

# # Augment training data
# X_train_aug = augment_emg(X_train)
# y_train_aug = y_train.copy()

# # Combine original + augmented
# X_train_final = np.vstack([X_train, X_train_aug])
# y_train_final = np.vstack([y_train, y_train_aug])

# print("New Training Shape:", X_train_final.shape)


Only for s1-s5, still overfitting.

Increasing data

In [ ]:
# from tensorflow.keras.callbacks import EarlyStopping

# model_final = Sequential()

# # CNN Block 1
# model_final.add(Conv1D(64, kernel_size=3, activation='relu', input_shape=(500, 10)))
# model_final.add(BatchNormalization())
# model_final.add(MaxPooling1D(pool_size=2))
# model_final.add(Dropout(0.4))  # Increased dropout

# # CNN Block 2
# model_final.add(Conv1D(128, kernel_size=4, activation='relu'))
# model_final.add(BatchNormalization())
# model_final.add(MaxPooling1D(pool_size=2))
# model_final.add(Dropout(0.4))

# # CNN Block 3
# model_final.add(Conv1D(256, kernel_size=5, activation='relu'))
# model_final.add(BatchNormalization())
# model_final.add(MaxPooling1D(pool_size=4))
# model_final.add(Dropout(0.4))

# # LSTM
# model_final.add(LSTM(128, dropout=0.4, recurrent_dropout=0.3))

# # Dense Layers
# model_final.add(Dense(128, activation='relu'))
# model_final.add(Dropout(0.5))
# model_final.add(Dense(10, activation='softmax'))

# # Compile
# model_final.compile(optimizer=Adam(learning_rate=0.0003),
#                     loss='categorical_crossentropy',
#                     metrics=['accuracy'])

# early_stop = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)

# # Train
# history_final = model_final.fit(
#     X_train_final, y_train_final,
#     validation_data=(X_test, y_test),
#     epochs=80,
#     batch_size=32,
#     callbacks=[early_stop]
# )
